In [145]:
# huffman encoding
# https://stepik.org/lesson/13239/step/5
from collections import Counter
from heapq import heapify, heappush, heappop
si = input()
#si = "beep-boop-beer!"
#si = "abacabad"
#si = "a"
#si = ""
cn = Counter(si)
#print(cn)
pq = [(kv[1], i, [kv[0]]) for i, kv in enumerate(cn.items())]
heapify(pq)
#print(pq)
while len(pq) > 1:
    a, b = heappop(pq), heappop(pq)
    heappush(pq, (a[0]+b[0], max(a[1],b[1]), [b[2],a[2]]))
    #print(pq)
if pq:
    bt = pq[0][2]
    cq = [("", bt)]
    en = {}
    while cq:
        cc, el = cq.pop()
        #print("-->", cc, "-->", el, "--", len(el))
        if len(el) == 1:
            en[el[0]] = cc
        else:
            cq.append((cc+"1", el[0]))
            cq.append((cc+"0", el[1]))
        #print(cq)
        #print(en)
    #print(bt)
    if len(bt) == 1:
        en[bt[0]] = "0"
se = "".join(en[ch] for ch in si)
print(len(cn), len(se))
print(*("{}: {}".format(ch, en[ch]) for ch, _ in cn.most_common()), sep="\n")
print(se)

[[['e'], [['p'], [['!'], ['r']]]], [[['-'], ['o']], ['b']]]
7 40
e: 11
b: 00
o: 010
-: 011
p: 101
r: 1000
!: 1001
0011111010110001001010101100111110001001


In [162]:
# huffman decoding (solution #1)
# https://stepik.org/lesson/13239/step/6

nc, sz = map(int, input().split())
en = dict(input().split(': ') for _ in range(nc))
se = input()

#nc, sz = 4, 14
#en = {'a':'0','b':'10','c':'110','d':'111'}
#se = '01001100100111'

#nc, sz = 7, 40
#en = {'e':'11','b':'00','o':'010','-':'011','p':'101','r':'1000','!':'1001'}
#se = '0011111010110001001010101100111110001001'

ht = [None, None]
zu = {'0':0, '1':1}
for ch, cc in en.items():
    ti = tp = ht
    for bc in cc:
        bc = zu[bc]
        if not ti[bc]:
            ti[bc] = [None, None]
        tp = ti
        ti = ti[bc]
    tp[bc] = ch
#print(ht)
so = []
ti = ht
for bc in se:
    bc = zu[bc]
    ti = ti[bc]
    if type(ti) == str:
        so.append(ti)
        ti = ht
print(''.join(so))

1 1
a: 0
0
a


In [167]:
# huffman decoding (solution #2)
# https://stepik.org/lesson/13239/step/6

nc, sz = map(int, input().split())
en = [input().split(': ') for _ in range(nc)]
se = input()

#nc, sz = 4, 14
#en = [('a','0'),('b','10'),('c','110'),('d','111')]
#se = '01001100100111'

#nc, sz = 7, 40
#en = [('e','11'),('b','00'),('o','010'),('-','011'),('p','101'),('r','1000'),('!','1001')]
#se = '0011111010110001001010101100111110001001'

so = ""
while se:
    for ch, bc in en:
        if se.startswith(bc):
            so += ch
            se = se[len(bc):]
print(so)

4 14
a: 0
b: 10
c: 110
d: 111
01001100100111
abacabad


In [ ]:
# huffman practice
# https://stepik.org/lesson/13245/step/1

In [208]:
%%writefile huffman.py
from collections import Counter, namedtuple
from heapq import heapify, heappush, heappop

class Node(namedtuple("Node", ["left", "right"])):
    def walk(self, code, acc):
        self.left.walk(code, acc + "0")
        self.right.walk(code, acc + "1")

class Leaf(namedtuple("Leaf", ["char"])):
    def walk(self, code, acc):
        code[self.char] = acc or "0"

def huffman_encode(s):
    heap = []
    for ch, freq in Counter(s).items():
        heap.append((freq, len(heap), Leaf(ch)))

    heapify(heap)

    count = len(heap)
    while len(heap) > 1:
        freq1, _count1, left = heappop(heap)
        freq2, _count2, right = heappop(heap)
        node = Node(left, right)
        heappush(heap, (freq1 + freq2, count, node))
        count += 1

    code = {}
    if heap:
        [(_freq, _count, root)] = heap
        root.walk(code, "")

    return code

def main():
    s = input()
    code = huffman_encode(s)
    encoded = "".join(code[ch] for ch in s)
    print(len(code), len(encoded))
    for ch in sorted(code):
        print("{}: {}".format(ch, code[ch]))
    print(encoded)

def huffman_decode(code, encoded):
    result = ""
    while encoded:
        for ch, seq in code.items():
            if encoded.startswith(seq):
                result += ch
                encoded = encoded[len(seq):]
                break
    return result

def test(n_iter=1000):
    import random
    import string
    
    for i in range(n_iter):
        length = random.randint(0, 32)
        s = "".join(random.choice(string.ascii_letters) for _ in range(length))
        code = huffman_encode(s)
        encoded = "".join(code[ch] for ch in s)
        assert huffman_decode(code, encoded) == s

    print("OK")

main()
#test()

Overwriting huffman.py


In [209]:
%%bash
python3 huffman.py <<< abcdabc

OK


In [193]:
%%bash
python3 huffman.py <<< abracadabra

5 23
a: 0
b: 110
c: 101
d: 100
r: 111
01101110101010001101110


In [194]:
%%bash
python3 huffman.py <<< a

1 1
a: 0
0


In [195]:
%%bash
python3 huffman.py <<< aaaaa

1 5
a: 0
00000


In [196]:
%%bash
python3 huffman.py <<< ""

0 0

